# Visualization EDepSim Hits

This notebook is used to demonstrate visualatioin tools and functionality.

To run this, you need the following:
  * (pre-pre-req) make sure the run `source setenv_py3.sh` or else environment variables and such are missing
  * run the simulation once and create the channel info file, `chmap.json`
  * run the simulation once and create an output file
  
The visualization tools utilize the python library [plotly](https://plotly.com/python/) and [plotly.go](https://plotly.com/python/graph-objects/).

You will need to install this

    pip3 install plotly
    
Other pre-req is `numpy`

    pip3 install numpy
    


In [ ]:
# Import the modules we'll use
import sys,os
edeptly_path = os.environ['EDEPTLY_DIR']
if edeptly_path not in sys.path:
    if os.path.exists(edeptly_path):
        print("Adding edeptly path: ",edeptly_path)
        sys.path.insert( 0, edeptly_path )
    else:
        print("Warning did not find edeptly path: ",edeptly_path)
import edeptly
import ROOT as rt

import plotly
import plotly.graph_objects as go
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
import cennsly.read_channel_json 
from cennsly.read_channel_json import read_channel_json
j = read_channel_json( "chmap.json" ) # run the sim first to get this file

The channel json file holds information that let's us draw different channel shapes.  Right now circle and square shapes are defined. The initial intention is to represent PMTs with circles and SiPMs with squares. (Though this is programmable by you in the different channel sdconfigs.)

In [ ]:
# Draw an example circle

import cennsly
from cennsly.draw_channels import gen_circle_mesh_channel

# find a circle channel
circle_ch = None
for ch in j["channeldata"]:
    if j["channeldata"][ch]["shape"]=="circle":
        circle_ch = int(ch)
        break
        
if circle_ch is not None:
    # if we found one, draw it
    # we choose a channel number and send the channel information
    #   and get back a 3D triangle mesh to plot
    vertices,faces = gen_circle_mesh_channel( circle_ch, j )
    print("printing channel ID=",circle_ch)
    print("vertex array shape: ",vertices.shape)
    print("face array shape: ",faces.shape)
    meshplot = go.Mesh3d(x=vertices[:,0],y=vertices[:,1],z=vertices[:,2],
                         i=faces[:,0],j=faces[:,1],k=faces[:,2],
                        color='red',
                        opacity=0.5)
    fig = go.Figure(data=meshplot)
    fig.show()
else:
    print("No circle channels defined.")

In [ ]:
# Draw an example square

import cennsly
from cennsly.draw_channels import gen_square_mesh_channel

# find a circle channel
sq_ch = None
for ch in j["channeldata"]:
    if j["channeldata"][ch]["shape"]=="square":
        sq_ch = int(ch)
        break
        
if sq_ch is not None:
    # if we found one, draw it
    # we choose a channel number and send the channel information
    #   and get back a 3D triangle mesh to plot
    vertices,faces = gen_square_mesh_channel( sq_ch, j )
    print("printing channel ID=",sq_ch)
    print(vertices.shape)
    print(faces.shape)
    #print(vertices)
    #print(faces)
    meshplot_square = go.Mesh3d(x=vertices[:,0],y=vertices[:,1],z=vertices[:,2],
                                i=faces[:,0],j=faces[:,1],k=faces[:,2],
                                color='red', opacity=0.5)
    fig = go.Figure(data=meshplot_square)
    fig.show()
else:
    print("No square channels defined.")

# Adding data

You'll need the output of the simulation.

In [ ]:
# Open file
input_file = "test.root"
rinput = rt.TFile(input_file)

# Get the TTree where we've stored all the data
edeptree = rinput.Get("EDepSimEvents")

# Get the number of entries
nentries = edeptree.GetEntries()
print("Entries in sim file: ",nentries)

In [ ]:
# Plot one event

edeptree.GetEntry(0)
event = edeptree.Event
detnames = edeptree.Event.GetSegmentDetectorNameList()
print("number of detnames: ",detnames.size())
for idet in range(detnames.size()):
    detname = detnames.at(idet)
    seghits_v = edeptree.Event.SegmentDetectors[detname]
    print("  Number of energy deposit in [",detname,"]: ",seghits_v.size())

#print("Number of energy deposit segments: ",nhits)


In [ ]:
import edeptly
from edeptly import draw_edepsim_points
hits = draw_edepsim_points( event )

fig1 = go.Figure(data=hits)
fig1.show()


## MC Truth in the output files

The Edep-sim machinery absorbed also stores primary particles and true particle trajectories from Geant.

In [ ]:
# Primaries: these are the particles given to geant4 to simulate

# e.g. the particle from single particle generators
# e.g. the final state particles from MARLEY
# e.g. the nuclear recoil from CEVENS generator

# Get the class holding truth energy deposit information
edeptree = rinput.Get("EDepSimEvents")

# Read an entry in the tree
edeptree.GetEntry(0)

nprimaries = edeptree.Event.Primaries.size()
print("Number of primaries: ",nprimaries)

for i in range(nprimaries):
    primary = edeptree.Event.Primaries.at(i)
    print("Primary Vertex [",i,"] //////////////  ")
    print(" Generator Name: ", primary.GeneratorName )
    print(" Reaction Name: ", primary.Reaction )
    print(" == particle-list =======")
    nparts = primary.Particles.size()
    for p in range(nparts):
        part = primary.Particles.at(p)
        mom = part.GetMomentum()
        print("    [",p,"]: PDG=",part.GetPDGCode(),
              " TrackId=",part.GetTrackId(),
             " P=(%.2f,%.2f,%.2f,%.2f)"%(mom[0],mom[1],mom[2],mom[3]))


In [ ]:
# Trajectories

ntrajs = edeptree.Event.Trajectories.size()
print("Number of trajectories: ",ntrajs)

print("/// Trajectories ////////")
for t in range(ntrajs):
    traj = edeptree.Event.Trajectories.at(t)
    print("  [",t,"] pdg=",traj.GetPDGCode(),
          " NPts=",traj.Points.size(),
          " TrackId=",traj.GetTrackId(),
          " ParentId=",traj.GetParentId())
    

In [ ]:
# Add these to the plots above
import cennsly
from cennsly.draw_edepsim_trajectories import draw_edepsim_event_trajectories

track_v = draw_edepsim_event_trajectories(edeptree.Event)
# Note: looks like we need refinements on saving trajectory steps
print("number of trajector plots: ",len(track_v))

for p in track_v:
    p["line"]["width"]=3.0
    
plot_track = circle_plots+[seg_plot]+track_v+cylinder_plot
#plot_track += square_plots_culled # if we have sipms


figseg1 = go.Figure(data=plot_track)
figseg1.update_layout(width=1000,height=600)
figseg1.show() 